In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_hotel       = pl.read_parquet(datadir / "hotel.parquet")
pl_customer    = pl.read_parquet(datadir / "customer.parquet")
pl_reservation = pl.read_parquet(datadir / "reservation.parquet")

# 5章 抽出
## 5-1 列名指定による列の選択


### Q: 集計対象の列のみに絞り込み


#### Awesome


In [ ]:
# 列名で指定して列を選択
pl_reservation.select(
    pl.col([
        "reservation_id",
        "hotel_id",
        "customer_id",
        "checkin_date",
        "checkout_date",
    ])
)

## 5-2 条件指定による列の抽出


### Q 列名が`tag`から始まる列の抽出

#### Awesome


In [ ]:
# 列の開始と終了の部分一致ルールを正規表現で指定して選択
pl_hotel.select(pl.col("^tag_.*$"))

### Q 欠損のある列の抽出


#### Awesome


In [ ]:
# （1）欠損を含む列の列名を取得
cols = [col.name for col in pl_customer.select(pl.all().is_null()) if col.any()]
# （2）列名を使って選択
pl_customer.select(cols)

### Q 数値型の列の抽出


#### Awesome


In [ ]:
# Int64型の列を選択
pl_reservation.select(pl.col(pl.Int64))

## 5-3 条件指定による行の抽出


### Q: 宿泊人数が2〜4人の予約履歴のみ抽出


#### Awesome

In [ ]:
# people_numが2から4の範囲の行を抽出
pl_reservation.filter(pl.col("people_num").is_between(2, 4))

## 5-4 ランダムサンプリング


### Q: 予約履歴をランダムサンプリング

#### Awesome


In [ ]:
# 20000件をランダムサンプリング
pl_reservation.sample(20000)

## 5-5 不均衡データの調整


### Q: 未キャンセルデータをキャンセル済データと同数になるようにアンダーサンプリング


#### Awesome


In [ ]:
# （1）未キャンセルデータを抽出
majority = pl_reservation.filter(pl.col("status") == "reserved")

# （2）キャンセル済データを抽出
minority = pl_reservation.filter(pl.col("status") == "canceled")

# （3）キャンセル済データとランダムサンプリングした未キャンセルデータを結合
pl.concat([
    minority,
    majority.sample(len(minority))
])